<a href="https://colab.research.google.com/github/gustavovazquez/ML/blob/main/ML_carga_dataframes_pandas_normalzacion_pipelines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Scikit-Learn - Carga de datos, normalización y uso de pipelines
Este notebook cubre operaciones esenciales para el análisis de datos en `scikit-learn` y `pandas`, incluyendo carga de datos, exploración, preprocesamiento con normalización y estandarización, cálculo de correlaciones y el uso de **pipelines** en Machine Learning.

Los **pipelines** son una herramienta clave para optimizar y estructurar el flujo de trabajo en Machine Learning, permitiendo aplicar pasos de preprocesamiento y modelado de manera eficiente.

Más sobre
- preprocesameinto: https://scikit-learn.org/stable/modules/preprocessing.html
- carga de datos: https://pandas.pydata.org/docs/getting_started/intro_tutorials/02_read_write.html
- pipelines: https://scikit-learn.org/stable/modules/compose.html


## **1. Cargar un archivo CSV en un dataframe de Pandas**
Para trabajar con datos en `scikit-learn`, utilizamos `pandas` para cargar datasets en formato CSV.

Si el archivo CSV está almacenado localmente, se puede cargar de la siguiente manera (la línea está comentada para que no genere error):

In [281]:
import pandas as pd

# Cargar un archivo CSV en un DataFrame
# df = pd.read_csv("ruta/del/archivo.csv")

Si el archivo CSV está disponible en línea (por ejemplo, en un repositorio de GitHub), se puede cargar directamente desde la URL del archivo en formato `raw`:

In [282]:
url = "https://raw.githubusercontent.com/gustavovazquez/datasets/main/heart.csv"
df = pd.read_csv(url)

## **2. Mostrar las primeras filas del dataframe**
Después de cargar los datos, es importante verificar que se haya importado correctamente. Para ello, podemos visualizar las primeras filas del dataset con:

In [283]:
# Mostrar tamaño y las primeras 5 filas
print("Tamaño del dataset:  filas", df.shape[0], "colmnas: ", df.shape[1])
df.head(10)

Tamaño del dataset:  filas 918 colmnas:  12


,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0
5,39,M,NAP,120,339,0,Normal,170,N,0.0,Up,0
6,45,F,ATA,130,237,0,Normal,170,N,0.0,Up,0
7,54,M,ATA,110,208,0,Normal,142,N,0.0,Up,0
8,37,M,ASY,140,207,0,Normal,130,Y,1.5,Flat,1
9,48,F,ATA,120,284,0,Normal,120,N,0.0,Up,0


## **3. Mostrar información básica del dataframe**
Para entender la estructura del dataset, es útil conocer la cantidad de filas, columnas, y los tipos de datos en cada columna. Para esto podemos usar:

In [284]:
# Mostrar información general del dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 918 entries, 0 to 917
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Age             918 non-null    int64  
 1   Sex             918 non-null    object 
 2   ChestPainType   918 non-null    object 
 3   RestingBP       918 non-null    int64  
 4   Cholesterol     918 non-null    int64  
 5   FastingBS       918 non-null    int64  
 6   RestingECG      918 non-null    object 
 7   MaxHR           918 non-null    int64  
 8   ExerciseAngina  918 non-null    object 
 9   Oldpeak         918 non-null    float64
 10  ST_Slope        918 non-null    object 
 11  HeartDisease    918 non-null    int64  
dtypes: float64(1), int64(6), object(5)
memory usage: 86.2+ KB


In [285]:
# Mostrar cantidad de filas y columnas
dimensiones = df.shape
print(f"El dataset tiene {dimensiones[0]} filas y {dimensiones[1]} columnas")

El dataset tiene 918 filas y 12 columnas


In [286]:
# Mostrar estadísticas básicas de las variables numéricas
df.describe()

,Age,RestingBP,Cholesterol,FastingBS,MaxHR,Oldpeak,HeartDisease
count,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000
mean,53.510893,132.396514,198.799564,0.233115,136.809368,0.887364,0.553377
std,9.432617,18.514154,109.384145,0.423046,25.460334,1.066570,0.497414
min,28.000000,0.000000,0.000000,0.000000,60.000000,-2.600000,0.000000
25%,47.000000,120.000000,173.250000,0.000000,120.000000,0.000000,0.000000
50%,54.000000,130.000000,223.000000,0.000000,138.000000,0.600000,1.000000
75%,60.000000,140.000000,267.000000,0.000000,156.000000,1.500000,1.000000
max,77.000000,200.000000,603.000000,1.000000,202.000000,6.200000,1.000000


## **4. Normalización y Estandarización**
Algunas variables pueden tener escalas muy diferentes (por ejemplo, ingresos en miles de dólares y edad en años). Para evitar que una variable con valores grandes domine el entrenamiento del modelo, se aplican técnicas de escalado:

- **Normalización (MinMaxScaler):** Escala los valores a un rango entre 0 y 1.
- **Estandarización (StandardScaler):** Centra los datos en 0 con desviación estándar de 1.

### **4.1 Normalización (MinMaxScaler)**
La normalización ajusta todas las características a un mismo rango, normalmente entre 0 y 1, lo cual es útil cuando los modelos de Machine Learning dependen de la magnitud de los valores.

In [287]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
df_numericas = df.select_dtypes(include=['number'])
df_normalizado = scaler.fit_transform(df_numericas)
print(pd.DataFrame(df_normalizado, columns=df_numericas.columns).head())

        Age  RestingBP  Cholesterol  FastingBS     MaxHR   Oldpeak  \
0  0.244898       0.70     0.479270        0.0  0.788732  0.295455   
1  0.428571       0.80     0.298507        0.0  0.676056  0.409091   
2  0.183673       0.65     0.469320        0.0  0.267606  0.295455   
3  0.408163       0.69     0.354892        0.0  0.338028  0.465909   
4  0.530612       0.75     0.323383        0.0  0.436620  0.295455   

   HeartDisease  
0           0.0  
1           1.0  
2           0.0  
3           1.0  
4           0.0  


### **4.2 Estandarización (StandardScaler)**
Transforma los datos para que tengan **media 0 y desviación estándar 1**, lo cual ayuda a mejorar la estabilidad numérica en algunos modelos. La transformación es lineal, por lo tanto no afecta a la distribución original de los datos. Tiene mejor información para indicar cuánto influye en la variable objetivo (si aumenta -o disminuye- 1 desvío estandar la variable influye de la misma manera en el target).

In [288]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
df_estandarizado = scaler.fit_transform(df_numericas)
print(pd.DataFrame(df_estandarizado, columns=df_numericas.columns).head())

        Age  RestingBP  Cholesterol  FastingBS     MaxHR   Oldpeak  \
0 -1.433140   0.410909     0.825070  -0.551341  1.382928 -0.832432   
1 -0.478484   1.491752    -0.171961  -0.551341  0.754157  0.105664   
2 -1.751359  -0.129513     0.770188  -0.551341 -1.525138 -0.832432   
3 -0.584556   0.302825     0.139040  -0.551341 -1.132156  0.574711   
4  0.051881   0.951331    -0.034755  -0.551341 -0.581981 -0.832432   

   HeartDisease  
0     -1.113115  
1      0.898380  
2     -1.113115  
3      0.898380  
4     -1.113115  


## **5. Uso de Pipelines en Scikit-Learn**
Los **pipelines** permiten encadenar múltiples pasos de preprocesamiento y modelado en un flujo estructurado. Esto facilita la gestión del flujo de trabajo en Machine Learning y ayuda a evitar problemas como **data leakage** (fuga de datos).

### **5.1 Ventajas de los Pipelines**
Los pipelines tienen múltiples ventajas:
- **Automatización:** Permiten aplicar preprocesamiento y entrenamiento de forma automática.
- **Reproducibilidad:** Se mantiene un flujo de trabajo estandarizado.
- **Evita fugas de datos:** Se asegura que el escalado o transformación de los datos se realice únicamente con los datos de entrenamiento.


### **5.2 Ejemplo de Pipeline**
En este ejemplo, construiremos un **pipeline** que incluye los siguientes pasos:
1. **Escalado de las variables numéricas** utilizando `MinMaxScaler`.
2. **Entrenamiento de un modelo de clasificación** (Regresión Logística).

Esto garantiza que cualquier transformación aplicada a los datos de entrenamiento también se aplique a los datos de prueba sin riesgo de fuga de información.

In [289]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Dividir datos en entrenamiento y prueba
X = df_numericas.drop(columns=["HeartDisease"]) # ya teníamos las columnas numéricas
# es muy IMPORTANTE sacar la variable objetivo (que también era numérica)
# se recomienda usar columns para no tener que indicar el axis
y = df['HeartDisease']  # Ajusta la variable objetivo según el dataset usado
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Definir el pipeline
pipeline = Pipeline([
    ('scaler', MinMaxScaler()),
    ('model', LogisticRegression())
])


La gran ventaja que tenemos es que aplicar todos los pasos para procesar el flujo de tareas (escalar y luego entrenar el modelo) lo hacemos directamente sobre el pipeline.

In [290]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('scaler', MinMaxScaler()), ('model', LogisticRegression())])

con el pipeline entrenado, cuando querramos predecir datos de la validación simplemente invocamos el método predict del pipeline.

In [291]:

# make prediction with pipeline
y_pred = pipeline.predict(X_test)

# puedo armar un dataframe para comparar "a ojo" los valores verdaderos (y_test) vs. predichos (y_pred)
tabla = pd.DataFrame({'y_test': y_test, 'y_pred': y_pred})

tabla

,y_test,y_pred
668,0,0
30,1,0
377,1,1
535,1,1
807,0,0
...,...,...
211,1,0
745,1,0
584,1,1
878,0,0


In [292]:
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))


[[53 24]
 [28 79]]
              precision    recall  f1-score   support

           0       0.65      0.69      0.67        77
           1       0.77      0.74      0.75       107

    accuracy                           0.72       184
   macro avg       0.71      0.71      0.71       184
weighted avg       0.72      0.72      0.72       184



## 6. Pipeline para tratamiento de datos numéricos y categóricos
En muchos problemas no solo tenemos variables numéricas
(ejemplo: superficie en m², número de baños, año de construcción),
sino también variables categóricas (ejemplo: tipo de propiedad, ciudad, color de la vivienda). Además las variables categóricas, una vez que se aplica la técnica de One Hot Encoding no necesitan ser escaladas. Usamos un `ColumnTransformer`, que permite aplicar distintas transformaciones a distintos tipos de columnas dentro de un mismo pipeline.



In [293]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split # Import train_test_split

# Definir variable objetivo y predictoras
target = "HeartDisease"
df_numericas = df.select_dtypes(include=["number"]).columns.drop(target)
df_categoricas = df.select_dtypes(include=["object", "category"]).columns

# Select the columns from the original dataframe
X = pd.concat([df[df_numericas], df[df_categoricas]], axis=1) #axis indica que lo hacemos a nivel columnas)
y = df[target]

# Split data into training and test sets *after* creating X with all columns
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Transformaciones para cada tipo
# (Aquí estaremos agregando una etapa adicional que es la de imputación de datos
# faltantes, que incluso es dependiente para cada tipo de variable).
# En este caso usaremos StandardScaler.
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(drop="first", handle_unknown="ignore"))
])

# ColumnTransformer: aplica transformaciones en paralelo
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, df_numericas),
        ("cat", categorical_transformer, df_categoricas)
    ]
)

# Pipeline final: preprocesamiento + modelo
pipeline  = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("classifier", LogisticRegression()) # Instantiated LogisticRegression
])

pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

In [294]:
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[67 10]
 [17 90]]
              precision    recall  f1-score   support

           0       0.80      0.87      0.83        77
           1       0.90      0.84      0.87       107

    accuracy                           0.85       184
   macro avg       0.85      0.86      0.85       184
weighted avg       0.86      0.85      0.85       184

